In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import NearMiss, RandomUnderSampler # for under_sampling
from imblearn.combine import SMOTETomek # for over_sampling
from imblearn.over_sampling import RandomOverSampler,ADASYN,SMOTE # for over_sampling



In [ ]:
# !pip install -U imbalanced-learn
# !pip install imblearn
# !pip install imblearn
# !pip install -c glemaitre imbalanced-learn
# !pip install imblearn==0.0


In [2]:
df = pd.read_csv("loan.csv")
df.sample(10)

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
132244,12000,36 months,6.03,NaN,MORTGAGE,78000.0,credit_card,UT,15.20,0.0,24.4,38.0,0,29.0,verified
74501,8000,36 months,14.09,10.0,MORTGAGE,29000.0,credit_card,TX,16.76,0.0,74.7,27.0,1,16.0,verified
19863,12000,36 months,6.54,1.0,RENT,33000.0,debt_consolidation,NC,14.84,0.0,42.9,33.0,0,18.0,verified
24573,20000,36 months,7.88,6.0,OWN,85000.0,credit_card,NY,12.99,0.0,34.1,26.0,0,22.0,verified
162465,5000,36 months,12.39,0.0,OWN,40000.0,major_purchase,GA,5.04,0.0,75.3,20.0,0,11.0,not verified
153345,28000,60 months,18.92,10.0,MORTGAGE,78000.0,home_improvement,SC,18.82,0.0,87.5,37.0,0,16.0,not verified
90435,10000,36 months,6.62,10.0,MORTGAGE,120000.0,debt_consolidation,TX,11.47,0.0,91.7,29.0,0,21.0,not verified
69848,5500,36 months,17.27,8.0,RENT,33000.0,debt_consolidation,HI,13.45,0.0,61.9,29.0,1,11.0,verified
128212,12800,60 months,12.99,10.0,RENT,75000.0,debt_consolidation,NY,14.26,0.0,43.6,42.0,0,20.0,verified
128076,10000,36 months,11.44,0.0,RENT,28000.0,debt_consolidation,MD,25.25,0.0,37.8,15.0,0,8.0,verified


In [3]:
df.head()

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
0,5000,36 months,10.65,10.0,RENT,24000.0,credit_card,AZ,27.65,0.0,83.7,9.0,0,26.0,verified
1,2500,60 months,15.27,0.0,RENT,30000.0,car,GA,1.00,0.0,9.4,4.0,1,12.0,verified
2,2400,36 months,15.96,10.0,RENT,12252.0,small_business,IL,8.72,0.0,98.5,10.0,0,10.0,not verified
3,10000,36 months,13.49,10.0,RENT,49200.0,other,CA,20.00,0.0,21.0,37.0,0,15.0,verified
4,5000,36 months,7.90,3.0,RENT,36000.0,wedding,AZ,11.20,0.0,28.3,12.0,0,7.0,verified


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163987 entries, 0 to 163986
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   loan_amnt              163987 non-null  int64  
 1   term                   163987 non-null  object 
 2   int_rate               163987 non-null  float64
 3   emp_length             158183 non-null  float64
 4   home_ownership         163987 non-null  object 
 5   annual_inc             163983 non-null  float64
 6   purpose                163987 non-null  object 
 7   addr_state             163987 non-null  object 
 8   dti                    163987 non-null  float64
 9   delinq_2yrs            163958 non-null  float64
 10  revol_util             163794 non-null  float64
 11  total_acc              163958 non-null  float64
 12  bad_loan               163987 non-null  int64  
 13  longest_credit_length  163958 non-null  float64
 14  verification_status    163987 non-nu

In [5]:
df.isna().sum()

loan_amnt                   0
term                        0
int_rate                    0
emp_length               5804
home_ownership              0
annual_inc                  4
purpose                     0
addr_state                  0
dti                         0
delinq_2yrs                29
revol_util                193
total_acc                  29
bad_loan                    0
longest_credit_length      29
verification_status         0
dtype: int64

In [6]:
df.describe()

,loan_amnt,int_rate,emp_length,annual_inc,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length
count,163987.000000,163987.000000,158183.000000,1.639830e+05,163987.000000,163958.000000,163794.000000,163958.000000,163987.000000,163958.000000
mean,13074.169141,13.715904,5.684353,7.191567e+04,15.881530,0.227357,54.079173,24.579734,0.183039,14.854274
std,7993.556189,4.391940,3.610664,5.907092e+04,7.587668,0.694168,25.285367,11.685190,0.386700,6.947733
min,500.000000,5.420000,0.000000,1.896000e+03,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,7000.000000,10.650000,2.000000,4.500000e+04,10.230000,0.000000,35.600000,16.000000,0.000000,10.000000
50%,11325.000000,13.490000,6.000000,6.100000e+04,15.620000,0.000000,55.800000,23.000000,0.000000,14.000000
75%,18000.000000,16.320000,10.000000,8.500000e+04,21.260000,0.000000,74.200000,31.000000,0.000000,18.000000
max,35000.000000,26.060000,10.000000,7.141778e+06,39.990000,29.000000,150.700000,118.000000,1.000000,65.000000


In [7]:
from pandas_profiling import ProfileReport


In [8]:
# pfile = ProfileReport(df)
# pfile.to_file(output_file= "badloan.html")

In [9]:
# import sweetviz as sv
# sweet_repo = sv.analyze(df)
# sweet_repo.show_html("Loan.html")

In [10]:
df.shape

(163987, 15)

In [11]:
df.columns

Index(['loan_amnt', 'term', 'int_rate', 'emp_length', 'home_ownership',
       'annual_inc', 'purpose', 'addr_state', 'dti', 'delinq_2yrs',
       'revol_util', 'total_acc', 'bad_loan', 'longest_credit_length',
       'verification_status'],
      dtype='object')

# preprocessing

### 1

In [12]:
df.loan_amnt.nunique()

1274

In [13]:
sns.boxplot(df["loan_amnt"])

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


<AxesSubplot:xlabel='loan_amnt'>

In [14]:
df[["loan_amnt"]].boxplot()

<AxesSubplot:xlabel='loan_amnt'>

### 2

In [15]:
df.term.nunique()

2

In [16]:
df.term.unique()

array(['36 months', '60 months'], dtype=object)

In [17]:
sns.countplot(df["term"])

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


<AxesSubplot:xlabel='term', ylabel='count'>

In [18]:
df["term"] = df["term"].apply(lambda x : x.split(" ")[0])

In [19]:
df["term"].dtype

dtype('O')

In [20]:
df["term"] = df["term"].astype(int)

In [21]:
df["term"].dtype

dtype('int32')

### 3

In [22]:
df.int_rate.nunique()

512

In [23]:
df[["int_rate"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

In [24]:
df["int_rate"].isna().sum()

0

In [25]:
q1 = df["int_rate"].quantile(0.25)
q3 = df["int_rate"].quantile(0.75)
IQR = q3-q1
upper_limit = q3 + 1.5*IQR
df["int_rate"] = np.where(df["int_rate"] > upper_limit,upper_limit,df["int_rate"])

In [26]:
df[["int_rate"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

### 4

In [27]:
df.emp_length.unique()

array([10.,  0.,  3.,  9.,  4.,  5.,  1.,  6.,  2.,  7.,  8., nan])

In [28]:
df.emp_length.nunique()

11

In [29]:
df["emp_length"].isna().sum()

5804

In [30]:
df["emp_length"].fillna(df["emp_length"].mean(),inplace = True)

In [31]:
df["emp_length"].isna().sum()

0

In [32]:
df[["emp_length"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

### 5

In [33]:
df.home_ownership.unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE', 'ANY'], dtype=object)

In [34]:
df.home_ownership.nunique()

6

In [35]:
df.home_ownership.isna().sum()

0

In [36]:
df.home_ownership.dtype

dtype('O')

In [37]:
df.home_ownership.value_counts().to_dict()

{'MORTGAGE': 79714,
 'RENT': 70526,
 'OWN': 13560,
 'OTHER': 156,
 'NONE': 30,
 'ANY': 1}

In [38]:
df["home_ownership"].replace({'MORTGAGE': 4,'RENT': 3,
                              'OWN': 5, 'OTHER': 2,
                              'NONE': 0,'ANY': 1},inplace = True)

In [39]:
df.home_ownership.dtype

dtype('int64')

In [40]:
df.annual_inc

0         24000.0
1         30000.0
2         12252.0
3         49200.0
4         36000.0
           ...   
163982    45000.0
163983    80000.0
163984    38000.0
163985    60000.0
163986    63078.0
Name: annual_inc, Length: 163987, dtype: float64

### 6

In [41]:
df.annual_inc.nunique()

14112

In [42]:
df.annual_inc.isna().sum()

4

In [43]:
df["annual_inc"].fillna(df["annual_inc"].mean(),inplace = True)

In [44]:
df.annual_inc.isna().sum()

0

In [45]:
df[["annual_inc"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

In [46]:
q1 = df["annual_inc"].quantile(0.25)
q3 = df["annual_inc"].quantile(0.75)
IQR = q3-q1
upper_limit = q3 + 1.5*IQR
df["annual_inc"] = np.where(df["annual_inc"] > upper_limit,upper_limit,df["annual_inc"])

In [47]:
df[["annual_inc"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

### 7

In [48]:
df.purpose

0                credit_card
1                        car
2             small_business
3                      other
4                    wedding
                 ...        
163982           credit_card
163983      home_improvement
163984    debt_consolidation
163985           credit_card
163986    debt_consolidation
Name: purpose, Length: 163987, dtype: object

In [49]:
df.purpose.nunique()

14

In [50]:
from sklearn.preprocessing import LabelEncoder

In [51]:
label_enc = LabelEncoder()
df["purpose"] = label_enc.fit_transform(df["purpose"])


In [52]:
df.purpose

0          1
1          0
2         11
3          9
4         13
          ..
163982     1
163983     4
163984     2
163985     1
163986     2
Name: purpose, Length: 163987, dtype: int32

In [53]:
df.purpose.nunique()

14

In [54]:
df.purpose.unique()

array([ 1,  0, 11,  9, 13,  2,  4,  6,  7,  8, 12,  5, 10,  3])

In [55]:
df.purpose.isna().sum()

0

### 8

In [56]:
df.addr_state

0         AZ
1         GA
2         IL
3         CA
4         AZ
          ..
163982    OK
163983    VA
163984    TX
163985    NY
163986    PA
Name: addr_state, Length: 163987, dtype: object

In [57]:
df.addr_state.nunique()

50

In [58]:
df.addr_state.value_counts().to_dict()

{'CA': 28702,
 'NY': 14285,
 'TX': 12128,
 'FL': 11396,
 'NJ': 6457,
 'IL': 6099,
 'PA': 5427,
 'VA': 5282,
 'GA': 5189,
 'OH': 4896,
 'NC': 4393,
 'MA': 4164,
 'AZ': 3941,
 'MD': 3890,
 'WA': 3823,
 'MI': 3762,
 'CO': 3531,
 'MN': 2771,
 'MO': 2627,
 'CT': 2563,
 'NV': 2387,
 'OR': 2149,
 'WI': 1980,
 'AL': 1965,
 'LA': 1885,
 'SC': 1826,
 'KY': 1477,
 'IN': 1463,
 'UT': 1389,
 'KS': 1335,
 'OK': 1331,
 'TN': 1327,
 'AR': 1104,
 'HI': 955,
 'NM': 839,
 'NH': 752,
 'WV': 714,
 'RI': 695,
 'DC': 584,
 'MT': 483,
 'DE': 445,
 'AK': 413,
 'WY': 376,
 'SD': 317,
 'VT': 270,
 'MS': 163,
 'NE': 13,
 'IA': 12,
 'ID': 9,
 'ME': 3}

In [59]:
label_enc = LabelEncoder()
df["addr_state"] = label_enc.fit_transform(df["addr_state"])


In [60]:
df.addr_state.value_counts().to_dict()

{4: 28702,
 33: 14285,
 42: 12128,
 9: 11396,
 30: 6457,
 14: 6099,
 37: 5427,
 44: 5282,
 10: 5189,
 34: 4896,
 27: 4393,
 19: 4164,
 3: 3941,
 20: 3890,
 46: 3823,
 22: 3762,
 5: 3531,
 23: 2771,
 24: 2627,
 6: 2563,
 32: 2387,
 36: 2149,
 47: 1980,
 1: 1965,
 18: 1885,
 39: 1826,
 17: 1477,
 15: 1463,
 43: 1389,
 16: 1335,
 35: 1331,
 41: 1327,
 2: 1104,
 11: 955,
 31: 839,
 29: 752,
 48: 714,
 38: 695,
 7: 584,
 26: 483,
 8: 445,
 0: 413,
 49: 376,
 40: 317,
 45: 270,
 25: 163,
 28: 13,
 12: 12,
 13: 9,
 21: 3}

In [61]:
df.addr_state.isna().sum()

0

### 9

In [62]:
df.dti

0         27.65
1          1.00
2          8.72
3         20.00
4         11.20
          ...  
163982    31.44
163983    23.65
163984     9.03
163985    10.10
163986    31.70
Name: dti, Length: 163987, dtype: float64

In [63]:
df.dti.isna().sum()

0

In [64]:
df[["dti"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

In [65]:
q1 = df["dti"].quantile(0.25)
q3 = df["dti"].quantile(0.75)
IQR = q3-q1
upper_limit = q3 + 1.5*IQR
df["dti"] = np.where(df["dti"] > upper_limit,upper_limit,df["dti"])

In [66]:
df[["dti"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

### 10

In [67]:
df.delinq_2yrs

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
163982    4.0
163983    0.0
163984    0.0
163985    1.0
163986    0.0
Name: delinq_2yrs, Length: 163987, dtype: float64

In [68]:
df.delinq_2yrs.isna().sum()

29

In [69]:
df.delinq_2yrs.value_counts()

0.0     139459
1.0      17158
2.0       4635
3.0       1488
4.0        579
5.0        310
6.0        144
7.0         68
8.0         42
9.0         26
12.0        14
10.0        12
11.0        10
14.0         4
18.0         3
13.0         2
15.0         2
29.0         1
16.0         1
Name: delinq_2yrs, dtype: int64

In [70]:
df["delinq_2yrs"] = df["delinq_2yrs"].fillna(16)

In [71]:
df.delinq_2yrs.value_counts()

0.0     139459
1.0      17158
2.0       4635
3.0       1488
4.0        579
5.0        310
6.0        144
7.0         68
8.0         42
16.0        30
9.0         26
12.0        14
10.0        12
11.0        10
14.0         4
18.0         3
13.0         2
15.0         2
29.0         1
Name: delinq_2yrs, dtype: int64

In [72]:
df[["delinq_2yrs"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

In [73]:
q1 = df["delinq_2yrs"].quantile(0.25)
q3 = df["delinq_2yrs"].quantile(0.75)
IQR = q3-q1
upper_limit = q3 + 1.5*IQR
df["delinq_2yrs"] = np.where(df["delinq_2yrs"] > upper_limit,upper_limit,df["delinq_2yrs"])

In [74]:
df[["delinq_2yrs"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

### 11

In [75]:
df.revol_util

0         83.7
1          9.4
2         98.5
3         21.0
4         28.3
          ... 
163982    75.8
163983    68.8
163984    70.7
163985    61.2
163986    54.0
Name: revol_util, Length: 163987, dtype: float64

In [76]:
df["revol_util"].nunique()

1170

In [77]:
df["revol_util"].isna().sum()

193

In [78]:
df["revol_util"] = df["revol_util"].fillna(df["revol_util"].mean())

In [79]:
df["revol_util"].isna().sum()

0

In [80]:
df[["revol_util"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

### 12

In [81]:
df.total_acc

0          9.0
1          4.0
2         10.0
3         37.0
4         12.0
          ... 
163982    34.0
163983    18.0
163984    24.0
163985    20.0
163986    28.0
Name: total_acc, Length: 163987, dtype: float64

In [82]:
df["total_acc"].nunique()

96

In [83]:
df["total_acc"].isna().sum()

29

In [84]:
df["total_acc"] = df["total_acc"].fillna(df["total_acc"].mean())

In [85]:
df["total_acc"].isna().sum()

0

In [86]:
df[["total_acc"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

In [87]:
q1 = df["total_acc"].quantile(0.25)
q3 = df["total_acc"].quantile(0.75)
IQR = q3-q1
upper_limit = q3 + 1.5*IQR
df["total_acc"] = np.where(df["total_acc"] > upper_limit,upper_limit,df["total_acc"])

In [88]:
df[["total_acc"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

### 13

In [89]:
df.bad_loan

0         0
1         1
2         0
3         0
4         0
         ..
163982    0
163983    0
163984    0
163985    0
163986    0
Name: bad_loan, Length: 163987, dtype: int64

In [90]:
df.bad_loan.isna().sum()

0

In [91]:
df.bad_loan.value_counts()

0    133971
1     30016
Name: bad_loan, dtype: int64

### 14

In [92]:
df["longest_credit_length"]

0         26.0
1         12.0
2         10.0
3         15.0
4          7.0
          ... 
163982    20.0
163983    22.0
163984     9.0
163985     6.0
163986    16.0
Name: longest_credit_length, Length: 163987, dtype: float64

In [93]:
df["longest_credit_length"].nunique()

63

In [94]:
df["longest_credit_length"].isna().sum()

29

In [95]:
df["longest_credit_length"] = df["longest_credit_length"].fillna(df["longest_credit_length"].mean())

In [96]:
df["longest_credit_length"].isna().sum()

0

In [97]:
df[["longest_credit_length"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

In [98]:
q1 = df["longest_credit_length"].quantile(0.25)
q3 = df["longest_credit_length"].quantile(0.75)
IQR = q3-q1
upper_limit = q3 + 1.5*IQR
df["longest_credit_length"] = np.where(df["longest_credit_length"] > upper_limit,upper_limit,df["longest_credit_length"])

In [99]:
df[["longest_credit_length"]].boxplot()

<AxesSubplot:xlabel='term', ylabel='count'>

### 15

In [100]:
df.verification_status

0             verified
1             verified
2         not verified
3             verified
4             verified
              ...     
163982    not verified
163983        verified
163984    not verified
163985        verified
163986        verified
Name: verification_status, Length: 163987, dtype: object

In [101]:
df.verification_status.nunique()

2

In [102]:
df.verification_status.isna().sum()

0

In [103]:
df.verification_status.value_counts().to_dict()

{'verified': 104832, 'not verified': 59155}

In [104]:
df["verification_status"].replace({'verified': 1, 'not verified': 0},inplace=True)

In [105]:
df.verification_status

0         1
1         1
2         0
3         1
4         1
         ..
163982    0
163983    1
163984    0
163985    1
163986    1
Name: verification_status, Length: 163987, dtype: int64

In [106]:
df

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
0,5000,36,10.65,10.0,3,24000.0,1,3,27.65,0.0,83.7,9.0,0,26.0,1
1,2500,60,15.27,0.0,3,30000.0,0,10,1.00,0.0,9.4,4.0,1,12.0,1
2,2400,36,15.96,10.0,3,12252.0,11,14,8.72,0.0,98.5,10.0,0,10.0,0
3,10000,36,13.49,10.0,3,49200.0,9,4,20.00,0.0,21.0,37.0,0,15.0,1
4,5000,36,7.90,3.0,3,36000.0,13,3,11.20,0.0,28.3,12.0,0,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163982,15000,60,12.39,3.0,4,45000.0,1,35,31.44,0.0,75.8,34.0,0,20.0,0
163983,20000,36,14.99,10.0,5,80000.0,4,44,23.65,0.0,68.8,18.0,0,22.0,1
163984,12825,36,17.14,6.0,4,38000.0,2,42,9.03,0.0,70.7,24.0,0,9.0,0
163985,27650,60,21.99,0.0,3,60000.0,1,33,10.10,0.0,61.2,20.0,0,6.0,1


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163987 entries, 0 to 163986
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   loan_amnt              163987 non-null  int64  
 1   term                   163987 non-null  int32  
 2   int_rate               163987 non-null  float64
 3   emp_length             163987 non-null  float64
 4   home_ownership         163987 non-null  int64  
 5   annual_inc             163987 non-null  float64
 6   purpose                163987 non-null  int32  
 7   addr_state             163987 non-null  int32  
 8   dti                    163987 non-null  float64
 9   delinq_2yrs            163987 non-null  float64
 10  revol_util             163987 non-null  float64
 11  total_acc              163987 non-null  float64
 12  bad_loan               163987 non-null  int64  
 13  longest_credit_length  163987 non-null  float64
 14  verification_status    163987 non-nu

# Feature selection

In [108]:
x = df.drop("bad_loan",axis = 1)
y = df["bad_loan"]

# spliting

In [109]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1,stratify=y)

In [110]:
y_train.value_counts()

0    107176
1     24013
Name: bad_loan, dtype: int64

In [111]:
y_train.value_counts(normalize=True)

0    0.816959
1    0.183041
Name: bad_loan, dtype: float64

In [112]:
y_test.value_counts()

0    26795
1     6003
Name: bad_loan, dtype: int64

In [113]:
y_test.value_counts(normalize=True)

0    0.816971
1    0.183029
Name: bad_loan, dtype: float64

# Model Building

In [114]:
rf_model = RandomForestClassifier(n_estimators=30,random_state=1)
rf_model.fit(x_train,y_train)

RandomForestClassifier(n_estimators=30, random_state=1)

# Test Acc

In [115]:
y_pred = rf_model.predict(x_test)
Accuracy = accuracy_score(y_test,y_pred)
print("Accuracy of Test Data:",Accuracy)
print()
cnf_mat  = confusion_matrix(y_test,y_pred)
print("Confusion Marix :\n",cnf_mat)
print()
clf_Rep = classification_report(y_test,y_pred)
print("Classification Repoet:\n",clf_Rep)

Accuracy of Test Data: 0.8160253674004513

Confusion Marix :
 [[26436   359]
 [ 5675   328]]

Classification Repoet:
               precision    recall  f1-score   support

           0       0.82      0.99      0.90     26795
           1       0.48      0.05      0.10      6003

    accuracy                           0.82     32798
   macro avg       0.65      0.52      0.50     32798
weighted avg       0.76      0.82      0.75     32798



# Oversampling and Undersampling

In [116]:
# !pip install -U imbalanced-learn
# !pip install imblearn
# !pip install imblearn
# !pip install -c glemaitre imbalanced-learn
# !pip install imblearn==0.0


## Undersampling

In [117]:
from imblearn.under_sampling import NearMiss, RandomUnderSampler

### 1

In [119]:
ns = NearMiss()
x_train_ns , y_train_ns = ns.fit_resample(x_train,y_train)
print(y_train.value_counts())
print(y_train_ns.value_counts())

0    107176
1     24013
Name: bad_loan, dtype: int64
0    24013
1    24013
Name: bad_loan, dtype: int64


In [121]:
rf_model = RandomForestClassifier(n_estimators=30,random_state=1)
rf_model.fit(x_train_ns,y_train_ns)

RandomForestClassifier(n_estimators=30, random_state=1)

# Test Acc

In [122]:
y_pred = rf_model.predict(x_test)
Accuracy = accuracy_score(y_test,y_pred)
print("Accuracy of Test Data:",Accuracy)
print()
cnf_mat  = confusion_matrix(y_test,y_pred)
print("Confusion Marix :\n",cnf_mat)
print()
clf_Rep = classification_report(y_test,y_pred)
print("Classification Repoet:\n",clf_Rep)

Accuracy of Test Data: 0.5251234831392158

Confusion Marix :
 [[13102 13693]
 [ 1882  4121]]

Classification Repoet:
               precision    recall  f1-score   support

           0       0.87      0.49      0.63     26795
           1       0.23      0.69      0.35      6003

    accuracy                           0.53     32798
   macro avg       0.55      0.59      0.49     32798
weighted avg       0.76      0.53      0.58     32798



### 2

In [124]:
ru = RandomUnderSampler()
x_train_ru , y_train_ru = ru.fit_resample(x_train,y_train)
print(y_train.value_counts())
print(y_train_ru.value_counts())

0    107176
1     24013
Name: bad_loan, dtype: int64
0    24013
1    24013
Name: bad_loan, dtype: int64


In [125]:
rf_model = RandomForestClassifier(n_estimators=30,random_state=1)
rf_model.fit(x_train_ru,y_train_ru)

RandomForestClassifier(n_estimators=30, random_state=1)

# Test Acc

In [126]:
y_pred = rf_model.predict(x_test)
Accuracy = accuracy_score(y_test,y_pred)
print("Accuracy of Test Data:",Accuracy)
print()
cnf_mat  = confusion_matrix(y_test,y_pred)
print("Confusion Marix :\n",cnf_mat)
print()
clf_Rep = classification_report(y_test,y_pred)
print("Classification Repoet:\n",clf_Rep)

Accuracy of Test Data: 0.6449173730105494

Confusion Marix :
 [[17465  9330]
 [ 2316  3687]]

Classification Repoet:
               precision    recall  f1-score   support

           0       0.88      0.65      0.75     26795
           1       0.28      0.61      0.39      6003

    accuracy                           0.64     32798
   macro avg       0.58      0.63      0.57     32798
weighted avg       0.77      0.64      0.68     32798



## Oversampling

In [127]:
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler,ADASYN,SMOTE

### 1

In [128]:
ro = RandomOverSampler()
x_train_ro , y_train_ro = ro.fit_resample(x_train,y_train)
print(y_train.value_counts())
print(y_train_ro.value_counts())

0    107176
1     24013
Name: bad_loan, dtype: int64
0    107176
1    107176
Name: bad_loan, dtype: int64


In [129]:
rf_model = RandomForestClassifier(n_estimators=30,random_state=1)
rf_model.fit(x_train_ro,y_train_ro)

RandomForestClassifier(n_estimators=30, random_state=1)

# Test Acc

In [130]:
y_pred = rf_model.predict(x_test)
Accuracy = accuracy_score(y_test,y_pred)
print("Accuracy of Test Data:",Accuracy)
print()
cnf_mat  = confusion_matrix(y_test,y_pred)
print("Confusion Marix :\n",cnf_mat)
print()
clf_Rep = classification_report(y_test,y_pred)
print("Classification Repoet:\n",clf_Rep)

Accuracy of Test Data: 0.8085249100554912

Confusion Marix :
 [[25722  1073]
 [ 5207   796]]

Classification Repoet:
               precision    recall  f1-score   support

           0       0.83      0.96      0.89     26795
           1       0.43      0.13      0.20      6003

    accuracy                           0.81     32798
   macro avg       0.63      0.55      0.55     32798
weighted avg       0.76      0.81      0.77     32798



## 2

In [131]:
ad = ADASYN()
x_train_ad , y_train_ad = ad.fit_resample(x_train,y_train)
print(y_train.value_counts())
print(y_train_ad.value_counts())

0    107176
1     24013
Name: bad_loan, dtype: int64
1    110151
0    107176
Name: bad_loan, dtype: int64


In [132]:
rf_model = RandomForestClassifier(n_estimators=30,random_state=1)
rf_model.fit(x_train_ad,y_train_ad)

RandomForestClassifier(n_estimators=30, random_state=1)

# Test Acc

In [133]:
y_pred = rf_model.predict(x_test)
Accuracy = accuracy_score(y_test,y_pred)
print("Accuracy of Test Data:",Accuracy)
print()
cnf_mat  = confusion_matrix(y_test,y_pred)
print("Confusion Marix :\n",cnf_mat)
print()
clf_Rep = classification_report(y_test,y_pred)
print("Classification Repoet:\n",clf_Rep)

Accuracy of Test Data: 0.773187389474968

Confusion Marix :
 [[24080  2715]
 [ 4724  1279]]

Classification Repoet:
               precision    recall  f1-score   support

           0       0.84      0.90      0.87     26795
           1       0.32      0.21      0.26      6003

    accuracy                           0.77     32798
   macro avg       0.58      0.56      0.56     32798
weighted avg       0.74      0.77      0.75     32798



## 3

In [134]:
sm = SMOTE()
x_train_sm , y_train_sm = sm.fit_resample(x_train,y_train)
print(y_train.value_counts())
print(y_train_sm.value_counts())

0    107176
1     24013
Name: bad_loan, dtype: int64
0    107176
1    107176
Name: bad_loan, dtype: int64


In [135]:
rf_model = RandomForestClassifier(n_estimators=30,random_state=1)
rf_model.fit(x_train_sm,y_train_sm)

RandomForestClassifier(n_estimators=30, random_state=1)

# Test Acc

In [136]:
y_pred = rf_model.predict(x_test)
Accuracy = accuracy_score(y_test,y_pred)
print("Accuracy of Test Data:",Accuracy)
print()
cnf_mat  = confusion_matrix(y_test,y_pred)
print("Confusion Marix :\n",cnf_mat)
print()
clf_Rep = classification_report(y_test,y_pred)
print("Classification Repoet:\n",clf_Rep)

Accuracy of Test Data: 0.7762058662113543

Confusion Marix :
 [[24191  2604]
 [ 4736  1267]]

Classification Repoet:
               precision    recall  f1-score   support

           0       0.84      0.90      0.87     26795
           1       0.33      0.21      0.26      6003

    accuracy                           0.78     32798
   macro avg       0.58      0.56      0.56     32798
weighted avg       0.74      0.78      0.76     32798



# Hyperparameter_ OPtimization

In [138]:
from sklearn.model_selection import RandomizedSearchCV

In [139]:
rf_model = RandomForestClassifier()
hyp = {"n_estimators" : np.arange(10,30),
       "criterion" : ["gini" , "entropy"] , 
       "min_samples_leaf" : np.arange(2,20) , 
       "min_samples_split" : np.arange(2,10) , 
       "random_state" : np.arange(10,30)}

In [140]:
rscv_rf_model = RandomizedSearchCV(rf_model , hyp , cv = 8)
rscv_rf_model.fit(x_train_ro,y_train_ro)

RandomizedSearchCV(cv=8, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'min_samples_leaf': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                                        'min_samples_split': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'n_estimators': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29]),
                                        'random_state': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29])})

In [141]:
rscv_rf_model.best_estimator_

RandomForestClassifier(criterion='entropy', min_samples_leaf=2,
                       min_samples_split=8, n_estimators=19, random_state=19)

In [142]:
rscv_rf_model.best_params_

{'random_state': 19,
 'n_estimators': 19,
 'min_samples_split': 8,
 'min_samples_leaf': 2,
 'criterion': 'entropy'}

# Test Acc

In [143]:
y_pred = rscv_rf_model.predict(x_test)
Accuracy = accuracy_score(y_test,y_pred)
print("Accuracy of Test Data:",Accuracy)
print()
cnf_mat  = confusion_matrix(y_test,y_pred)
print("Confusion Marix :\n",cnf_mat)
print()
clf_Rep = classification_report(y_test,y_pred)
print("Classification Repoet:\n",clf_Rep)

Accuracy of Test Data: 0.7862064760046344

Confusion Marix :
 [[24391  2404]
 [ 4608  1395]]

Classification Repoet:
               precision    recall  f1-score   support

           0       0.84      0.91      0.87     26795
           1       0.37      0.23      0.28      6003

    accuracy                           0.79     32798
   macro avg       0.60      0.57      0.58     32798
weighted avg       0.75      0.79      0.77     32798



In [123]:
'loan_amnt',
'term', 
'int_rate', 
'emp_length', 
'home_ownership',
'annual_inc', 
'purpose', 
'addr_state', 
'dti', 
'delinq_2yrs',
'revol_util',
'total_acc', 
'bad_loan', 
'longest_credit_length',
'verification_status'

'verification_status'